In [76]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [77]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
    if mel:
                mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
                result=np.hstack((result, mel))
    return result

In [78]:
#DataFlair - Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#DataFlair - Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [79]:
#DataFlair - Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("ser_dataset\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [80]:
#DataFlair - Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [81]:
#DataFlair - Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [82]:
#DataFlair - Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [83]:
#DataFlair - Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [84]:
#DataFlair - Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [85]:
#DataFlair - Predict for the test set
y_pred=model.predict(x_test)

In [86]:
#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 76.04%


In [87]:
x_train

array([[-5.22061890e+02,  3.50668907e+01,  3.75342917e+00, ...,
         1.65243153e-04,  1.04321596e-04,  6.55571566e-05],
       [-6.41227722e+02,  4.49487762e+01, -1.85174108e+00, ...,
         3.89261913e-05,  3.05255344e-05,  2.94166621e-05],
       [-6.50705750e+02,  5.30211639e+01, -4.92040396e+00, ...,
         4.75216839e-05,  3.46632551e-05,  1.62844444e-05],
       ...,
       [-5.50096191e+02,  1.70297680e+01, -1.14575634e+01, ...,
         1.51764631e-04,  1.16828531e-04,  8.47479314e-05],
       [-5.55357605e+02,  4.71569710e+01,  1.10750732e+01, ...,
         1.61086471e-04,  1.04962477e-04,  6.52811796e-05],
       [-5.04816345e+02,  3.53618660e+01, -1.43495779e+01, ...,
         6.08151488e-04,  5.55269769e-04,  4.47782222e-04]])

In [88]:
y_pred

array(['happy', 'calm', 'happy', 'happy', 'disgust', 'calm', 'disgust',
       'disgust', 'disgust', 'happy', 'happy', 'disgust', 'fearful',
       'happy', 'disgust', 'happy', 'calm', 'disgust', 'happy', 'disgust',
       'disgust', 'disgust', 'disgust', 'calm', 'happy', 'happy',
       'disgust', 'happy', 'calm', 'fearful', 'happy', 'disgust', 'happy',
       'fearful', 'disgust', 'calm', 'calm', 'fearful', 'calm', 'disgust',
       'happy', 'disgust', 'calm', 'calm', 'disgust', 'disgust',
       'disgust', 'happy', 'calm', 'happy', 'fearful', 'fearful',
       'disgust', 'happy', 'happy', 'happy', 'disgust', 'happy', 'calm',
       'calm', 'disgust', 'calm', 'happy', 'calm', 'happy', 'calm',
       'calm', 'disgust', 'disgust', 'happy', 'happy', 'fearful',
       'fearful', 'fearful', 'fearful', 'disgust', 'fearful', 'happy',
       'calm', 'fearful', 'disgust', 'calm', 'fearful', 'calm', 'disgust',
       'disgust', 'disgust', 'happy', 'happy', 'fearful', 'disgust',
       'disgust

In [89]:
from joblib import dump, load

dump(model, 'emotion_model.joblib')

['emotion_model.joblib']

In [90]:
loaded_model = load('emotion_model.joblib')

In [91]:
p = loaded_model.predict(x_test)

#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=p)

#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))


Accuracy: 76.04%


In [97]:
import IPython.display as ipd



In [102]:
x_test[0]

array([-6.01385986e+02,  6.33781929e+01, -1.15760441e+01,  2.18321743e+01,
        2.16284871e+00, -4.25698709e+00,  9.03013945e-01, -6.61770391e+00,
       -1.79809868e+00,  2.75439763e+00,  3.42172474e-01,  1.23036027e+00,
        1.70965886e+00, -4.90207858e-02,  7.42392421e-01, -2.06753087e+00,
        1.01961404e-01,  2.84777355e+00, -2.35306311e+00,  9.85819638e-01,
       -6.73846662e-01, -3.52432132e-01, -1.10208631e+00,  8.85100663e-01,
       -5.82186095e-02, -2.77087390e-01, -5.28626025e-01,  1.46822679e+00,
       -6.95827067e-01,  6.84519112e-01, -2.04375243e+00, -4.82658684e-01,
        1.29247546e-01, -4.06054318e-01,  4.52576369e-01,  6.76289260e-01,
       -6.57189786e-01,  5.65326750e-01,  1.41317666e-01, -7.85537481e-01,
        6.91090524e-01,  7.17880130e-01,  7.11319566e-01,  7.47310221e-01,
        7.36255705e-01,  7.57716894e-01,  7.66564250e-01,  7.93796957e-01,
        7.65873849e-01,  7.77435362e-01,  7.27934062e-01,  7.54741728e-01,
        1.00065954e-03,  

In [104]:
import librosa
import soundfile
import numpy as np
from joblib import load

# Function to extract features (mfcc) from a sound file
def extract_feature(file_name, mfcc=True):
  with soundfile.SoundFile(file_name) as sound_file:
    X = sound_file.read(dtype="float32")
    sample_rate = sound_file.samplerate
    if mfcc:
      mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
      return mfccs

# Load the saved model
model = load('emotion_model.joblib')  # Adjust filename if different

# Specify your audio file path
audio_file_path = "ser_dataset\\Actor_04\\03-01-01-01-01-01-04.wav"  # Replace with your audio file path

# Extract features from the audio file
features = extract_feature(audio_file_path)

# Reshape the features to match the model's input format (assuming 2D array)
features = np.expand_dims(features, axis=0)  # Add a new dimension for a single sample

# Predict the emotion
predicted_emotion = model.predict(features)[0]  # Get the first element from prediction

# Print the predicted emotion
print("Predicted Emotion:", predicted_emotion)


ValueError: X has 40 features, but MLPClassifier is expecting 180 features as input.